In [1]:
import sys
sys.path.insert(0, r"C:\Users\michellexu\Pulse\engine\src\python\pulse\rl-hemorrhage-resuscitation\gating")

from torch import nn
import os
import torch
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import joblib

In [2]:
script_dir = script_dir = os.getcwd()
parent_dir = os.path.dirname(script_dir)

data = pd.read_csv(os.path.join(script_dir, "train_data_bv.csv"))
data["dbv1"] = data["bv1"] - data["bv2"]
data["dbv2"] = data["bv2"] - data["bv3"]
filtered = data[(data["severity"] != 0.35) & (data["severity"] != 0.15)]
data = filtered
data

,bv1,bv2,bv3,severity,label,dbv1,dbv2
0,4672.020119,4661.731009,4651.516970,0.05,0,10.289111,10.214038
1,4672.020119,4649.813507,4627.959100,0.10,0,22.206613,21.854407
3,4672.020119,4652.312035,4633.303977,0.50,0,19.708085,19.008058
4,4672.020119,4646.909993,4622.921291,0.60,0,25.110126,23.988702
5,4672.020119,4641.411392,4612.436327,0.70,0,30.608727,28.975065
...,...,...,...,...,...,...,...
2094,3755.683265,3694.985034,3637.396574,0.25,1,60.698232,57.588460
2095,3755.683265,3680.153955,3609.281895,0.30,1,75.529310,70.872060
2097,3755.683265,3648.306231,3550.103007,0.40,1,107.377035,98.203224
2098,3755.683265,3718.991026,3684.734201,0.90,1,36.692239,34.256825


In [3]:
X_np = data.iloc[:, 5:].to_numpy()
y_np = data.iloc[:, 4].to_numpy()
X_np[0]

array([10.2891108 , 10.21403818])

In [4]:
X_np

array([[ 10.2891108 ,  10.21403818],
       [ 22.20661258,  21.85440654],
       [ 19.70808453,  19.00805761],
       ...,
       [107.37703453,  98.20322359],
       [ 36.69223895,  34.25682481],
       [ 40.44317647,  37.5161412 ]], shape=(1800, 2))

In [5]:
y_np

array([0, 0, 0, ..., 1, 1, 1], shape=(1800,))

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(X_np, y_np, test_size=0.3, random_state=42, shuffle=True, stratify=y_np)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp)

X_train_np = X_train
X_val_np = X_val
X_test_np = X_test

# fit scaler on training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_np)

# apply same transformation to val/test
X_val_scaled = scaler.transform(X_val_np)
X_test_scaled = scaler.transform(X_test_np)

# optionally save scaler for later use (inverse transform / inference)
joblib.dump(scaler, os.path.join(script_dir, "gating_scaler.pkl"))

# convert to torch tensors
X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class GatingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=2, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=32)
        self.fc3 = nn.Linear(in_features=32, out_features=16)
        self.fc4 = nn.Linear(in_features=16, out_features=2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

model = GatingNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 50
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    total = 0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        total += 1
        # if i % 5 == 1:    # print every 2 mini-batches
        #     print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 5:.3f}")
        #     running_loss = 0.0
    avg_train_loss = running_loss / total
    #train_acc = correct / total

    # validation pass
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            #inputs = inputs.to(device)
            #labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total if total > 0 else 0.0
    print(f"Finished epoch #{epoch + 1} train_loss: {avg_train_loss:.4f} val_loss: {avg_val_loss:.4f} val_acc: {val_acc:.4f}")


save_path = os.path.join(script_dir, "gating_model.pth")
torch.save(model.state_dict(), save_path)
print("Finished Training and saved model at", save_path)

Finished epoch #1 train_loss: 0.6858 val_loss: 0.6497 val_acc: 0.5000
Finished epoch #2 train_loss: 0.6434 val_loss: 0.6035 val_acc: 0.5741
Finished epoch #3 train_loss: 0.5992 val_loss: 0.5491 val_acc: 0.9222
Finished epoch #4 train_loss: 0.5455 val_loss: 0.4801 val_acc: 0.9593
Finished epoch #5 train_loss: 0.4791 val_loss: 0.4011 val_acc: 0.9704
Finished epoch #6 train_loss: 0.4043 val_loss: 0.3193 val_acc: 0.9741
Finished epoch #7 train_loss: 0.3276 val_loss: 0.2447 val_acc: 0.9741
Finished epoch #8 train_loss: 0.2584 val_loss: 0.1836 val_acc: 0.9704
Finished epoch #9 train_loss: 0.2022 val_loss: 0.1406 val_acc: 0.9741
Finished epoch #10 train_loss: 0.1635 val_loss: 0.1109 val_acc: 0.9741
Finished epoch #11 train_loss: 0.1360 val_loss: 0.0909 val_acc: 0.9741
Finished epoch #12 train_loss: 0.1179 val_loss: 0.0790 val_acc: 0.9741
Finished epoch #13 train_loss: 0.1082 val_loss: 0.0722 val_acc: 0.9741
Finished epoch #14 train_loss: 0.1006 val_loss: 0.0675 val_acc: 0.9667
Finished epoch 

In [7]:
# python
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np

# load data (adjust path as needed)
script_dir = os.getcwd()
data = pd.read_csv(os.path.join(script_dir, "train_data_bv.csv"))
data["dbv1"] = data["bv1"] - data["bv2"]
data["dbv2"] = data["bv2"] - data["bv3"]
data = data[data["severity"] != 0.35]

# features / labels: adjust column indices/names to your dataset
X = data.iloc[:, 5:].to_numpy()        # all feature columns from col index 5
y = data.iloc[:, 4].to_numpy().astype(int)  # label column at index 4

# Make sure you actually have the rows you expect
assert X.shape[0] == y.shape[0], "Feature / label row mismatch"

# split
X_train_np, X_temp, y_train_np, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y
)
X_val_np, X_test_np, y_val_np, y_test_np = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp
)

# scale: fit only on train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_np)
X_val_scaled = scaler.transform(X_val_np)
X_test_scaled = scaler.transform(X_test_np)

# to tensors
X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.long)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val = torch.tensor(y_val_np, dtype=torch.long)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)

# model - use actual input dimension
class GatingNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_features=input_dim, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=32)
        self.fc3 = nn.Linear(in_features=32, out_features=16)
        self.fc4 = nn.Linear(in_features=16, out_features=2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

input_dim = X_train.shape[1]
model = GatingNet(input_dim=input_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# training loop
n_epochs = 50
for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_train_loss = train_loss / total
    train_acc = correct / total

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / total if total > 0 else 0.0
    val_acc = correct / total if total > 0 else 0.0

    print(f"Epoch {epoch+1}/{n_epochs} train_loss={avg_train_loss:.4f} train_acc={train_acc:.4f} val_loss={avg_val_loss:.4f} val_acc={val_acc:.4f}")

# save model
torch.save(model.state_dict(), os.path.join(script_dir, "gating_model_fixed.pth"))

Epoch 1/50 train_loss=0.5870 train_acc=0.8718 val_loss=0.4660 val_acc=0.9075
Epoch 2/50 train_loss=0.3714 train_acc=0.9465 val_loss=0.2745 val_acc=0.9144
Epoch 3/50 train_loss=0.1946 train_acc=0.9421 val_loss=0.1669 val_acc=0.9144
Epoch 4/50 train_loss=0.1218 train_acc=0.9458 val_loss=0.1516 val_acc=0.9212
Epoch 5/50 train_loss=0.1100 train_acc=0.9421 val_loss=0.1551 val_acc=0.9247
Epoch 6/50 train_loss=0.1053 train_acc=0.9465 val_loss=0.1531 val_acc=0.9212
Epoch 7/50 train_loss=0.1067 train_acc=0.9407 val_loss=0.1497 val_acc=0.9178
Epoch 8/50 train_loss=0.1056 train_acc=0.9451 val_loss=0.1496 val_acc=0.9247
Epoch 9/50 train_loss=0.1045 train_acc=0.9473 val_loss=0.1518 val_acc=0.9212
Epoch 10/50 train_loss=0.1020 train_acc=0.9480 val_loss=0.1518 val_acc=0.9247
Epoch 11/50 train_loss=0.1036 train_acc=0.9458 val_loss=0.1495 val_acc=0.9212
Epoch 12/50 train_loss=0.1027 train_acc=0.9495 val_loss=0.1531 val_acc=0.9247
Epoch 13/50 train_loss=0.1024 train_acc=0.9502 val_loss=0.1480 val_acc=0.